In [1]:
import nltk

from nltk.corpus import brown

## 一元标注 unigram tagging 
## 为单个词分配标注 

In [2]:
brown_tagged_sents = brown.tagged_sents(categories='news')

brown_sents = brown.sents(categories='news')

#下面语句 称为训练（train）过程
unigram_tagger = nltk.UnigramTagger(brown_tagged_sents)

In [3]:
unigram_tagger.tag(brown_sents[2007])
#查看第2007个句子的标注

[('Various', 'JJ'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('apartments', 'NNS'),
 ('are', 'BER'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('terrace', 'NN'),
 ('type', 'NN'),
 (',', ','),
 ('being', 'BEG'),
 ('on', 'IN'),
 ('the', 'AT'),
 ('ground', 'NN'),
 ('floor', 'NN'),
 ('so', 'QL'),
 ('that', 'CS'),
 ('entrance', 'NN'),
 ('is', 'BEZ'),
 ('direct', 'JJ'),
 ('.', '.')]

In [4]:
unigram_tagger.evaluate(brown_tagged_sents)
#评估

0.9349006503968017

In [5]:
size = int(len(brown_tagged_sents)*0.9)
size

4160

In [6]:
train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]
#90% train
#10% test
unigram_tagger = nltk.UnigramTagger(train_sents)
unigram_tagger.evaluate(test_sents)

0.8121200039868434

In [ ]:
#n元标注 即使用前面的n-1的单词的标注来预测当前词的标注

In [7]:
#二元标注
bigram_tagger = nltk.BigramTagger(train_sents)#训练过程

bigram_tagger.tag(brown_sents[2007])

[('Various', 'JJ'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('apartments', 'NNS'),
 ('are', 'BER'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('terrace', 'NN'),
 ('type', 'NN'),
 (',', ','),
 ('being', 'BEG'),
 ('on', 'IN'),
 ('the', 'AT'),
 ('ground', 'NN'),
 ('floor', 'NN'),
 ('so', 'CS'),
 ('that', 'CS'),
 ('entrance', 'NN'),
 ('is', 'BEZ'),
 ('direct', 'JJ'),
 ('.', '.')]

In [8]:
unseen_sent = brown_sents[4203]

bigram_tagger.tag(unseen_sent)

#没有见过million前面出现过None
#所以预测million的标注失败了
#导致后续的都失败的
#正确率很低

[('The', 'AT'),
 ('population', 'NN'),
 ('of', 'IN'),
 ('the', 'AT'),
 ('Congo', 'NP'),
 ('is', 'BEZ'),
 ('13.5', None),
 ('million', None),
 (',', None),
 ('divided', None),
 ('into', None),
 ('at', None),
 ('least', None),
 ('seven', None),
 ('major', None),
 ('``', None),
 ('culture', None),
 ('clusters', None),
 ("''", None),
 ('and', None),
 ('innumerable', None),
 ('tribes', None),
 ('speaking', None),
 ('400', None),
 ('separate', None),
 ('dialects', None),
 ('.', None)]

In [9]:
bigram_tagger.evaluate(test_sents)

0.10206319146815508

In [10]:
#使用组合起来的标注器
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents , backoff=t0)
t2 = nltk.BigramTagger(train_sents , backoff=t1)

t2.evaluate(test_sents)

0.8452108043456593

In [11]:
t3 = nltk.TrigramTagger(train_sents , backoff=t2)

t3.evaluate(test_sents)

0.843317053722715

In [ ]:
nltk.BigramTagger(sents , cutoff=2 , backoff=t1)
#cutoff参数 丢弃只看到一次或两次的上下文

In [13]:
#将标注器（训练好的）保存起来
from pickle import dump

output = open('t2.pkl' , 'wb')

dump(t2 , output , -1)
output.close()

In [14]:
#将保存的标注器读入
from pickle import load

input_ = open('t2.pkl' , 'rb')

tagger = load(input_)
input_.close()

In [15]:
text = 'this is an apple'

tokens = text.split()

tagger.tag(tokens)

[('this', 'DT'), ('is', 'BEZ'), ('an', 'AT'), ('apple', 'NN')]

In [18]:
cfd = nltk.ConditionalFreqDist(
 ((x[1], y[1], z[0]), z[1])
 for sent in brown_tagged_sents
 for x, y, z in nltk.trigrams(sent))

In [19]:
ambiguous_contexts = [c for c in cfd.conditions() if len(cfd[c])>1]

In [20]:
sum(cfd[c].N() for c in ambiguous_contexts)/cfd.N()

0.049297702068029296

In [21]:
#标注句子的第一个词的时候 使用前一个句子的最后面的词
#但这些词语通常没有关系
#使用已标注句子集合进行训练
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)

train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents , backoff=t0)
t2 = nltk.BigramTagger(train_sents , backoff=t1)


In [22]:
t2.evaluate(test_sents)

0.8452108043456593

AttributeError: module 'nltk.tag.brill' has no attribute 'demo'

In [24]:
#基于转换的标注
#brill标注